# Importing Packages and Dataset

In [1]:
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [2]:
# Importing train datasets
client_train = pd.read_csv("./cleaned_data/cleaned_client_train.csv")
invoice_train = pd.read_csv("./cleaned_data/cleaned_invoice_train.csv")

display(client_train.head())
display(invoice_train.head())

,district,client_id,client_catg,region,creation_date,fraud_status
0,60,train_Client_0,11,101,1994-12-31,0.0
1,69,train_Client_1,11,107,2002-05-29,0.0
2,62,train_Client_10,11,301,1986-03-13,0.0
3,69,train_Client_100,11,105,1996-07-11,0.0
4,62,train_Client_1000,11,303,2014-10-14,0.0


,client_id,invoice_date,meter_number,meter_status,meter_code,reading_remark,meter_coefficient,consumption_level_1,consumption_level_2,consumption_level_3,consumption_level_4,old_index,months_number,meter_type,total_consumption,diff_in_index,is_index_discrepancy
0,train_Client_0,2014-03-24,1335667,0.0,203,8,1,82,0,0,0,14302,4,0,82,82,False
1,train_Client_0,2013-03-29,1335667,0.0,203,6,1,1200,184,0,0,12294,4,0,1384,1384,False
2,train_Client_0,2015-03-23,1335667,0.0,203,8,1,123,0,0,0,14624,4,0,123,123,False
3,train_Client_0,2015-07-13,1335667,0.0,207,8,1,102,0,0,0,14747,4,0,102,102,False
4,train_Client_0,2016-11-17,1335667,0.0,207,9,1,572,0,0,0,15066,12,0,572,572,False


# Feature Engineering

In [3]:
# Function to encode categorical variables and aggregate the values
def encode_categorical_variables(df):

    # Encoding categorical variables
    df = pd.get_dummies(df, columns=['meter_status', 'meter_coefficient', 'reading_remark', 'meter_type', 'is_index_discrepancy'])

    # Group by client_id and sum the values
    df = df.groupby('client_id').sum().reset_index()

    return df

# Function to aggregate categorical variables
def aggregate_categorical_vars(df):

    # Aggregating categorical variables
    agg_categorical_invoice_train = df.groupby('client_id').agg({
        'meter_number': 'nunique',
        'meter_code': 'nunique'
    })

    agg_categorical_invoice_train.columns = ['meter_number_count', 'meter_code_count']
    agg_categorical_invoice_train.reset_index(inplace=True)

    return agg_categorical_invoice_train

# Function to aggregate numerical variables
def aggregate_numerical_vars(df):

    # Aggregating numerical variables
    agg_invoice_train = df.groupby('client_id').agg({
        'consumption_level_1': ['mean', 'std', 'min', 'max', 'median', 'sum'],
        'consumption_level_2': ['mean', 'std', 'min', 'max', 'median', 'sum'],
        'consumption_level_3': ['mean', 'std', 'min', 'max', 'median', 'sum'],
        'consumption_level_4': ['mean', 'std', 'min', 'max', 'median', 'sum'],
        'old_index': ['mean', 'std', 'min', 'max', 'median'],
        'diff_in_index': ['mean', 'std', 'min', 'max', 'median', 'sum'],
        'total_consumption': ['mean', 'std', 'min', 'max', 'median', 'sum'],
        'months_number': ['mean', 'std', 'min', 'max', 'median']
    })

    agg_invoice_train.columns = ['{}_{}'.format(col, stat) for col, stat in agg_invoice_train.columns]
    agg_invoice_train.reset_index(inplace=True)

    return agg_invoice_train

# Function to get no. of invoices per client
def get_invoice_counts(df):

    # No. of invoices per client
    invoice_count = df.groupby('client_id').size().reset_index(name='no_of_invoices')

    return invoice_count

# Function to transform datetime features with aggregation
def transform_datetime(df):

    # Transform datetime formats
    df['invoice_date'] = pd.to_datetime(df['invoice_date'])

    # Sort the invoice_train dataset by invoice_date and clients
    temp_df = df.sort_values(by=['client_id', 'invoice_date'])
    temp_df = temp_df[['client_id', 'invoice_date']]

    # Calculate the difference between the invoice dates
    temp_df['time_since_last_invoice'] = temp_df.groupby('client_id')['invoice_date'].diff().dt.days.fillna(0)

    # Add columns of invoice year, month and day
    temp_df['invoice_year'] = temp_df['invoice_date'].dt.year
    temp_df['invoice_month'] = temp_df['invoice_date'].dt.month
    temp_df['invoice_day'] = temp_df['invoice_date'].dt.day

    # Create a helper function to get the second smallest time since last invoice as there are 0s
    def second_min(series):
        unique_vals = series.dropna().unique()  # Drop NaNs and get unique values
        unique_vals.sort()  # Sort the values
        if len(unique_vals) > 1:  # Check if there are at least two unique values
            return unique_vals[1]  # Return the second smallest
        else:
            return None  # Return None if there's no second smallest value

    # Aggregate the invoice dates
    agg_invoice_dates = temp_df.groupby('client_id').agg({
        'time_since_last_invoice': ['mean', 'std', second_min, 'max', 'median'],
        'invoice_year': ['mean', 'std', 'min', 'max', 'median'],
        'invoice_month': ['mean', 'std', 'min', 'max', 'median'],
        'invoice_day': ['mean', 'std', 'min', 'max', 'median']
    })

    # Rename the columns
    agg_invoice_dates.columns = ['{}_{}'.format(col, stat) for col, stat in agg_invoice_dates.columns]
    agg_invoice_dates.reset_index(inplace=True)

    return agg_invoice_dates

In [4]:
def feature_engineering(client_data, invoice_data):

    # transform the client_data dataset
    transformed_client_data = client_data.copy()

    # Transform the datetime format
    transformed_client_data['creation_date'] = pd.to_datetime(transformed_client_data['creation_date'])

    # Add columns of creation year, month and day
    transformed_client_data['creation_year'] = transformed_client_data['creation_date'].dt.year
    transformed_client_data['creation_month'] = transformed_client_data['creation_date'].dt.month
    transformed_client_data['creation_day'] = transformed_client_data['creation_date'].dt.day

    categorical_invoices = invoice_data[['client_id', 'meter_status', 'meter_number', 'meter_code', 'reading_remark', 'meter_coefficient', 'meter_type', 'is_index_discrepancy']]

    agg_numerical_invoice = aggregate_numerical_vars(invoice_data)
    agg_categorical_invoice = aggregate_categorical_vars(categorical_invoices)
    invoice_count = get_invoice_counts(invoice_data)
    agg_invoice_dates = transform_datetime(invoice_data)
    categorical_invoice_data = encode_categorical_variables(categorical_invoices)

    # Merge the datasets
    final_data = transformed_client_data.merge(agg_numerical_invoice, on='client_id', how='left')
    final_data = final_data.merge(agg_categorical_invoice, on='client_id', how='left')
    final_data = final_data.merge(invoice_count, on='client_id', how='left')
    final_data = final_data.merge(agg_invoice_dates, on='client_id', how='left')
    final_data = final_data.merge(categorical_invoice_data, on='client_id', how='left')

    # Set all NaN values to 0
    final_data.fillna(0, inplace=True)

    return final_data

In [5]:
final_dataset = feature_engineering(client_train, invoice_train)

display(final_dataset.head())

print(final_dataset.columns)

,district,client_id,client_catg,region,creation_date,fraud_status,creation_year,creation_month,creation_day,consumption_level_1_mean,...,meter_coefficient_40,meter_coefficient_50,reading_remark_6,reading_remark_7,reading_remark_8,reading_remark_9,meter_type_0,meter_type_1,is_index_discrepancy_False,is_index_discrepancy_True
0,60,train_Client_0,11,101,1994-12-31,0.0,1994,12,31,352.400000,...,0.0,0.0,21.0,0.0,8.0,6.0,35.0,0.0,35.0,0.0
1,69,train_Client_1,11,107,2002-05-29,0.0,2002,5,29,557.540541,...,0.0,0.0,20.0,0.0,6.0,11.0,37.0,0.0,37.0,0.0
2,62,train_Client_10,11,301,1986-03-13,0.0,1986,3,13,798.611111,...,0.0,0.0,10.0,0.0,5.0,3.0,18.0,0.0,18.0,0.0
3,69,train_Client_100,11,105,1996-07-11,0.0,1996,7,11,1.200000,...,0.0,0.0,19.0,0.0,0.0,1.0,20.0,0.0,20.0,0.0
4,62,train_Client_1000,11,303,2014-10-14,0.0,2014,10,14,663.714286,...,0.0,0.0,0.0,0.0,2.0,12.0,14.0,0.0,14.0,0.0


Index(['district', 'client_id', 'client_catg', 'region', 'creation_date',
       'fraud_status', 'creation_year', 'creation_month', 'creation_day',
       'consumption_level_1_mean',
       ...
       'meter_coefficient_40', 'meter_coefficient_50', 'reading_remark_6',
       'reading_remark_7', 'reading_remark_8', 'reading_remark_9',
       'meter_type_0', 'meter_type_1', 'is_index_discrepancy_False',
       'is_index_discrepancy_True'],
      dtype='object', length=105)


# Train Test Split

In [6]:
# Split the dataset into train and test
train_set, test_set = train_test_split(final_dataset, test_size=0.2, random_state=42)

print(train_set.shape)
print(test_set.shape)

(108394, 105)
(27099, 105)


In [8]:
# Save the final datasets
final_dataset.to_csv("./transformed_data/final_dataset.csv", index=False)
train_set.to_csv("./transformed_data/train_set.csv", index=False)
test_set.to_csv("./transformed_data/test_set.csv", index=False)

# Resampling

In [9]:
# Split the train_data into features and target
X = train_set.drop(['client_id', 'creation_date','fraud_status'], axis=1)
y = train_set['fraud_status']

## SMOTE

In [10]:
# Stratified K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
smote = SMOTE(random_state=42)

for i, (_, fold_index) in enumerate(skf.split(X, y)):
    print(f"Fold {i+1}: ")

    X_fold = X.iloc[fold_index]
    y_fold = y.iloc[fold_index]

    print(f"length of fold set: {len(X_fold)}")
    print(f"Percentage of fraud cases in train set: {y_fold.sum()/len(y)}")

    # Apply SMOTE
    X_fold_resampled, y_fold_resampled = smote.fit_resample(X_fold, y_fold)

    # Check the number of fraud cases after resampling
    print(f"Percentage of fraud cases in resampled fold set: {y_fold_resampled.sum()/len(y_fold_resampled)}")
    print(f"New length of fold set: {len(X_fold_resampled)}")

    # Combine fold data
    fold_data = pd.concat([X_fold, y_fold], axis=1)
    
    # Combine smote_fold data
    smote_fold_data = pd.concat([X_fold_resampled, y_fold_resampled], axis=1)

    # Save the data
    fold_data.to_csv(f"./smote_train_data/fold_{i+1}.csv", index=False)
    print("Validation data saved successfully!")
    smote_fold_data.to_csv(f"./smote_train_data/smote_fold_{i+1}.csv", index=False)
    print("Resampled train data saved successfully!")

    print("\n")

Fold 1: 
length of fold set: 21679
Percentage of fraud cases in train set: 0.011070723471778881
Percentage of fraud cases in resampled fold set: 0.5
New length of fold set: 40958
Validation data saved successfully!
Resampled train data saved successfully!


Fold 2: 
length of fold set: 21679
Percentage of fraud cases in train set: 0.011070723471778881
Percentage of fraud cases in resampled fold set: 0.5
New length of fold set: 40958
Validation data saved successfully!
Resampled train data saved successfully!


Fold 3: 
length of fold set: 21679
Percentage of fraud cases in train set: 0.01107994907467203
Percentage of fraud cases in resampled fold set: 0.5
New length of fold set: 40956
Validation data saved successfully!
Resampled train data saved successfully!


Fold 4: 
length of fold set: 21679
Percentage of fraud cases in train set: 0.01107994907467203
Percentage of fraud cases in resampled fold set: 0.5
New length of fold set: 40956
Validation data saved successfully!
Resampled tra

## Undersampling

In [11]:
rus = RandomUnderSampler(random_state=42)

print(f"Percentage of fraud cases in train set: {y.sum()/len(y)}")

undersampled_train_X, undersampled_train_y = rus.fit_resample(X, y)

print(f"Percentage of fraud cases in undersampled train set: {undersampled_train_y.sum()/len(undersampled_train_y)}")

undersampled_train_data = pd.concat([undersampled_train_X, undersampled_train_y], axis=1)

Percentage of fraud cases in train set: 0.0553720685646807
Percentage of fraud cases in undersampled train set: 0.5


In [12]:
# Save the undersampled data
undersampled_train_data.to_csv("./undersampled_train_data/undersampled_train.csv", index=False)